In [1]:
import csv
import pandas as pd
import numpy as np
import torch

In [24]:
#Load in the data and create dictionaries of the bitcoin ticker values
filename = 'Coin Prices.csv'
coin_prices_df = pd.read_csv(filename)
coin_dictionary = pd.Series(coin_prices_df.Ticker.values, index=coin_prices_df.index.values).to_dict()

filename = 'Coin Sentiment.csv'
sentiment_df = pd.read_csv(filename)
sentiment_dictionary = pd.Series(sentiment_df.Ticker.values, index=sentiment_df.index.values).to_dict()

In [25]:
#Process data into array of values and exclude specific coins
#List of coin tickers to ignore because of incomplete data sets:
coins_to_remove = ['BQX', 'IOTA', 'LLT', 'YOYO', 'VIBE']
coin_dictionary = {k: v for k, v in coin_dictionary.items() if not v in coins_to_remove}

price_data_list = []
sentiment_data_list = []
for coin in coin_dictionary.values():
    coin_price_list = coin_prices_df[coin_prices_df['Ticker'] == coin].values[0,5:]
    sentiment_list = sentiment_df[coin_prices_df['Ticker'] == coin].values[0,5:]
    price_data_list.append(coin_price_list)
    sentiment_data_list.append(sentiment_list)

In [26]:
#Only use the Coin Price Information At This Time
#Length of time series
L = len(price_data_list[0])
#Number of coins
N = len(coin_dictionary)

x = np.empty((N, L), 'float64')
x2 = np.empty((N, L), 'float64')
print(x.shape)

(100, 1287)


In [27]:
#Put all the data into the numpy array
for i in range(len(price_data_list)):
    x[i,:] = price_data_list[i]
    x2[i,:] = sentiment_data_list[i]

#Normalize the data for training
normalized_x =  x/np.linalg.norm(x, ord=np.inf, axis=1, keepdims=True)
#normalized_x2 =  x2/np.linalg.norm(x2, ord=np.inf, axis=1, keepdims=True)
normalized_x2 = x2

torch.save(x, open('x.pt', 'wb'))
torch.save(x2, open('x2.pt', 'wb'))

torch.save(normalized_x, open('coin_train.pt', 'wb'))
torch.save(normalized_x2, open('sentiment_train.pt', 'wb'))

In [45]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd

class Sequence(nn.Module):
    
    def __init__(self, input_nodes):
        super(Sequence, self).__init__()
        
        self.input_nodes = input_nodes
        self.output_nodes = 1
        
        self.number_of_nodes_1 = 13
        self.number_of_nodes_2 = 7
        self.number_of_nodes_3 = 4
        
        self.lstm1 = nn.LSTMCell(self.input_nodes, self.number_of_nodes_1)
        self.lstm2 = nn.LSTMCell(self.number_of_nodes_1, self.number_of_nodes_2)
        self.lstm3 = nn.LSTMCell(self.number_of_nodes_2, self.number_of_nodes_3)
        self.linear = nn.Linear(self.number_of_nodes_3, self.output_nodes)

    def forward(self, input, future = 0):
        outputs = []
        
        h_t = torch.zeros(input.size(0), self.number_of_nodes_1, dtype=torch.double)
        c_t = torch.zeros(input.size(0), self.number_of_nodes_1, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), self.number_of_nodes_2, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), self.number_of_nodes_2, dtype=torch.double)
        h_t3 = torch.zeros(input.size(0), self.number_of_nodes_3, dtype=torch.double)
        c_t3 = torch.zeros(input.size(0), self.number_of_nodes_3, dtype=torch.double)

        #Break the data up into chunks where we can determine the number of input nodes
        number_of_chunks = input.shape[1]-(self.input_nodes - 2)
        for i in range(number_of_chunks//2):
            input_t = input[:,2*i:2*i+self.input_nodes]
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            output = self.linear(h_t3)
            outputs += [output]
            
        for i in range(future):# if we should predict the future
            #Output is a single point but we need to feed more into the model
            output = outputs[-self.input_nodes:]
            output = torch.stack(output, 1).squeeze(2)
            
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            output = self.linear(h_t3)
            outputs += [output]
            
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [46]:
data = torch.load('coin_train.pt')
sentiment = torch.load('sentiment_train.pt')
x = torch.load('x.pt')
x_2 = torch.load('x2.pt')

#Zip together the data and sentiment 
joined = np.empty((x.shape[0], x.shape[1]*2), 'float64')
#target = np.empty((x.shape[0], x.shape[1]*2), 'float64')

for i in range(x.shape[1]):
    joined[:,2*i] = data[:,i]
    joined[:,2*i+1] = sentiment[:,i]

#Number of input points
num_vars = 2
input_nodes = 24 * num_vars

input = torch.from_numpy(joined[:, :-num_vars])
test_input = torch.from_numpy(joined[:3, :-num_vars])
target = torch.from_numpy(data[:, input_nodes//num_vars:])
test_target = torch.from_numpy(data[:3, input_nodes//num_vars:])

In [47]:
print(data.shape)
print(input.shape, target.shape, test_input.shape, test_target.shape)

(100, 1287)
torch.Size([100, 2572]) torch.Size([100, 1263]) torch.Size([3, 2572]) torch.Size([3, 1263])


In [48]:
def closure():
    optimizer.zero_grad()
    #Output predicts the single next value
    out = seq(input)
    loss = criterion(out, target)
    print('loss:', loss.item())
    loss.backward()
    return loss

def draw(yi, color):
    plt.plot(np.arange(target.size(1)), yi[:target.size(1)], color, linewidth = 2.0)
    plt.plot(np.arange(target.size(1), target.size(1) + future), yi[target.size(1):], color + ':', linewidth = 2.0)

def iteration_plot(y):
    # draw the result
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')
    plt.savefig('output/predict_iteration_%d.pdf'%i)
    plt.close()

In [49]:
iterations = 5

# build the model
seq = Sequence(input_nodes)
seq.double()
criterion = nn.MSELoss()

# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.25)

#begin to train
for i in range(iterations):
    
    print('STEP: ', i)
    optimizer.step(closure)
    
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.detach().numpy()
    
    # draw the result
    iteration_plot(y)

STEP:  0
loss: 0.46497702700079185
loss: 0.42712069998571606
loss: 0.24922282289574982
loss: 0.14245267859026753
loss: 0.0902713483936927
loss: 0.06553558214864048
loss: 0.053236308829387465
loss: 0.046833892897296246
loss: 0.04337851077866328
loss: 0.04144796366130546
loss: 0.04032368406708146
loss: 0.03963058869117774
loss: 0.039167521117227856
loss: 0.03882189462923986
loss: 0.03852653582122375
loss: 0.03823519645171928
loss: 0.03790597478994974
loss: 0.037494083408905525
loss: 0.036944685828118176
loss: 0.036107342766274426
test loss: 0.016433710354581754
STEP:  1
loss: 0.034688515740440885
loss: 0.03202016671244985
loss: 0.03393751066099999
loss: 0.03000581487778544
loss: 0.026524065365246645
loss: 0.024056589697947713
loss: 0.022112239623862134
loss: 0.02064556599050335
loss: 0.01970081349016439
loss: 0.01911433186437921
loss: 0.01872814914784023
loss: 0.018453865618117293
loss: 0.01824384898508225
loss: 0.018062833984126614
loss: 0.017889736644462792
loss: 0.017711155775375426
l

In [50]:
with open('bitcoin.pickle', 'wb') as handle:
    pickle.dump(seq, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('bitcoin.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [51]:
#Create Predictions for all coins
with torch.no_grad():
    future = 1000
    pred = seq(input, future=future)
    loss = criterion(pred[:, :-future], target)
    y = pred.detach().numpy()

In [52]:
#Transform the predictions back to the coin values
pred_transformed = pred.numpy() * np.linalg.norm(x, ord=np.inf, axis=1, keepdims=True)

In [53]:
#Find the values for specific time periods to create a nice tabular output
last_actuals = pred_transformed[:,target.shape[1]]
predicted_24_hrs = pred_transformed[:,target.shape[1] + 24]
predicted_48_hrs = pred_transformed[:,target.shape[1] + 48]
predicted_168_hrs = pred_transformed[:,target.shape[1] + 168]
predicted_24_percent = predicted_24_hrs / last_actuals
predicted_48_percent = predicted_48_hrs / last_actuals
predicted_168_percent = predicted_168_hrs / last_actuals

In [54]:
#Create a summary dataframe that shows the information
result_df = pd.DataFrame()
result_df['ticker'] = coin_dictionary.values()
result_df['Current_Val'] = last_actuals
result_df['24_hr_pred'] = predicted_24_hrs
result_df['48_hr_pred'] = predicted_48_hrs
result_df['168_hr_pred'] = predicted_168_hrs
result_df['24_hr_percent'] = predicted_24_percent
result_df['48_hr_percent'] = predicted_48_percent
result_df['168_hr_percent'] = predicted_168_percent

In [55]:
def draw_figures(preds, df, future=0, label=""):
    #Create a figure for each ticker
    for i, ticker in df['ticker'].iteritems():
        # draw the result
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        ticker_plot = preds[i]
        plt.plot(np.arange(len(ticker_plot)-future), ticker_plot[:len(ticker_plot)-future], 'r',
                 linewidth = 2.0,label=ticker)
        plt.plot(np.arange(len(ticker_plot)-1000, len(ticker_plot)), ticker_plot[len(ticker_plot)-future:],
                 'r:', linewidth = 2.0)
        plt.savefig('output/' + label + ticker + '.pdf')
        plt.close()

def draw_all_figures(preds, df, future=0, label=""):
    #Set up the initial plot
    fig = plt.figure(figsize=(30,10))
    plt.title('Predict future values for all coins\n(Dashlines are predicted values)', fontsize=30)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    ax = fig.add_subplot(111)
    cycler = ax._get_lines.prop_cycler
    
    for i, ticker in df['ticker'].iteritems():
        # draw a line for each prediction
        ticker_plot = preds[i]
        current_color = next(cycler)
        plt.plot(np.arange(len(ticker_plot)-future), ticker_plot[:len(ticker_plot)-future], current_color['color'], 
                 linewidth = 2.0, label=ticker)
        plt.plot(np.arange(len(ticker_plot)-1000, len(ticker_plot)), ticker_plot[len(ticker_plot)-future:], 
                 current_color['color'], linestyle = '--', linewidth = 2.0)
    
    plt.legend(loc='upper right')
    plt.savefig('output/All_' + label + '.pdf')
    plt.close()

In [56]:
#Predicted Top Performers
top_df = result_df.sort_values('24_hr_percent', ascending=False)[0:10]
#Create a graph of the top 10 performers
draw_figures(pred_transformed, top_df, 1000, "top_performers_")
draw_all_figures(pred_transformed, top_df, 1000, "top_performers")

top_df.style.format({
    '24_hr_percent': '{:.2%}'.format,
    '48_hr_percent': '{:.2%}'.format,
    '168_hr_percent': '{:.2%}'.format
})

C:\Users\bbiseda\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


,ticker,Current_Val,24_hr_pred,48_hr_pred,168_hr_pred,24_hr_percent,48_hr_percent,168_hr_percent
25,CDT,0.0087264,0.012054,0.0131026,0.0139997,138.13%,150.15%,160.43%
41,GVT,23.4845,29.5303,33.997,42.7338,125.74%,144.76%,181.97%
51,LTC,161.531,199.69,227.662,270.972,123.62%,140.94%,167.75%
57,MCO,6.80635,8.26993,9.18227,14.7664,121.50%,134.91%,216.95%
65,OMG,11.9923,14.5355,16.7515,22.4655,121.21%,139.69%,187.33%
34,EOS,5.68763,6.72379,6.50334,3.86234,118.22%,114.34%,67.91%
54,MTL,3.52806,4.12509,4.53756,7.51974,116.92%,128.61%,213.14%
95,WAVES,5.24963,6.00917,6.76781,10.3966,114.47%,128.92%,198.04%
28,MANA,0.0784562,0.0897583,0.0983361,0.165232,114.41%,125.34%,210.60%
27,DASH,512.303,586.028,661.512,1007.18,114.39%,129.12%,196.60%


In [57]:
#Predicted Worst Performers
bottom_df = result_df.sort_values('24_hr_percent', ascending=True)[0:10]
#Create a graph of the bottom 10 performers
draw_figures(pred_transformed, bottom_df, 1000, "bottom_performers_")
draw_all_figures(pred_transformed, bottom_df, 1000, "bottom_performers")

bottom_df.style.format({
    '24_hr_percent': '{:.2%}'.format,
    '48_hr_percent': '{:.2%}'.format,
    '168_hr_percent': '{:.2%}'.format
})

C:\Users\bbiseda\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


,ticker,Current_Val,24_hr_pred,48_hr_pred,168_hr_pred,24_hr_percent,48_hr_percent,168_hr_percent
84,SUB,0.743535,0.448711,0.425991,0.164636,60.35%,57.29%,22.14%
10,BAT,0.200413,0.122203,0.0924479,0.0565312,60.98%,46.13%,28.21%
68,PPT,15.64,11.0555,9.22893,6.32729,70.69%,59.01%,40.46%
5,AST,0.290615,0.20576,0.167705,0.117074,70.80%,57.71%,40.28%
52,LRC,0.294487,0.20927,0.1753,0.117973,71.06%,59.53%,40.06%
17,BTS,0.14147,0.100708,0.081768,0.0462155,71.19%,57.80%,32.67%
47,ICN,0.0169018,0.012125,0.0114671,0.00432695,71.74%,67.85%,25.60%
36,ETC,24.7415,17.8693,19.4423,16.0838,72.22%,78.58%,65.01%
23,LINK,0.526854,0.386067,0.355308,0.13433,73.28%,67.44%,25.50%
99,ZRX,0.482816,0.356713,0.29212,0.181413,73.88%,60.50%,37.57%
